<a href="https://colab.research.google.com/github/11bender/alumni-scraping/blob/main/alumni_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Linkedin scraper**

## Load libraries

In [ ]:
import os, random, sys, time 
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 

In [ ]:
browser = webdriver.Chrome('/driver/chromedriver')

Log in to your account at this step

In [ ]:
browser.get("https://www.linkedin.com/login/")

Get the school page


In [ ]:
browser.get("https://www.linkedin.com/school/ku_leuven/people/?educationEndYear=2020&educationStartYear=2012&keywords=statistics")

Scroll down to get all alumni

In [ ]:
rep = 100 # determine the rep enough to scroll all the page
last_height = browser.execute_script("return document.body.scrollHeight")

for i in range(rep):
  browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
  time.sleep(5)
  new_height = browser.execute_script("return document.body.scrollHeight")
  if new_height == last_height:
    break
  new_height = last_height

Get a list of profile url

In [ ]:
src = browser.page_source
soup = BeautifulSoup(src, 'lxml')
pav = soup.find('div', {'class' : 'scaffold-finite-scroll__content'})
all_links = pav.find_all('a', {'class' : "app-aware-link"})
profiles = [link.get("href") for link in all_links]
urls = [profile.split("?")[0] for profile in profiles]
urls = list(set(urls))
len(urls)

In [ ]:
stat_url = pd.DataFrame(urls, columns=['url'])
stat_url.to_csv('stat_url.csv')

Get data from each profile

In [ ]:
names = []
positions = []
companies = []
work_locs = []
work_times = []
colleges = []
degrees = []
college_times  = []
urls = []

for url in urls:
  browser.get(url)
  time.sleep(3)
  src = browser.page_source
  soup = BeautifulSoup(src, 'html.parser')
  
  #scraping name
  #name = soup.find('title').text.split("(23)")[1].split("| LinkedIn")[0].strip()
  name = soup.find('title').get_text().split("|")[0].strip()
  names.append(name)
  print(name)
  
  #scraping from Experience
  exp_url = os.path.join(url, "details/experience")
  print(exp_url)
  browser.get(exp_url)
  time.sleep(8)
  src = browser.page_source
  exp_soup = BeautifulSoup(src, 'html.parser')
  # company
  company_info = exp_soup.find_all('span', {"class": "t-14 t-normal"})
  company = [info.find("span", {"aria-hidden": "true"}).get_text() for info in company_info]
  companies.append(company)
  # position
  pos_info = exp_soup.find_all('span', {"class": "mr1 t-bold"})
  position = [info.find("span", {"aria-hidden": "true"}).get_text() for info in pos_info]
  positions.append(position)
  # location and time
  loc_time_info = exp_soup.find_all('span', {"class": "t-14 t-normal t-black--light"})
  work_time = [loc_time.find("span").get_text() for loc_time in loc_time_info if ' · ' in loc_time.find("span").get_text()]
  loc = list(set([loc_time.find("span").get_text() for loc_time in loc_time_info if ' · ' not in loc_time.find("span").get_text()]))
  work_locs.append(loc)
  work_times.append(work_time)

  #scraping from Education
  edu_url = os.path.join(url, "details/education")
  browser.get(edu_url)
  time.sleep(8)
  src = browser.page_source
  edu_soup = BeautifulSoup(src, 'html.parser')
  # college
  college_info = edu_soup.find_all('span', {'class':"mr1 hoverable-link-text t-bold"})
  college = [info.find("span", {"aria-hidden": "true"}).get_text() for info in college_info]
  colleges.append(college)
  # degree
  degree_info = edu_soup.find_all('span', {'class':"t-14 t-normal"})
  degree = [info.find("span", {"aria-hidden": "true"}).get_text() for info in degree_info]
  degrees.append(degree)
  # college-time
  college_time_info = edu_soup.find_all('span', {'class':"t-14 t-normal t-black--light"})
  college_time = [info.find("span", {"aria-hidden": "true"}).get_text() for info in college_time_info]
  college_times.append(college_time)

In [ ]:
data = pd.DataFrame(list(zip(names, urls, companies, positions, work_locs, work_times, colleges, degrees, college_times)), 
               columns =["names", "url", "companies", "positions", "work_locs", "work_times", "colleges", "degrees", "college_times"]) 

In [ ]:
data.to_csv('linkedin.csv', index=False)